In [1]:
from dingo import HPolytope
import numpy as np
from dingo.gurobi_based_implementations import fast_inner_ball
from dingo import PolytopeSampler
from polytope_oracles.membership import Membership
from polytope import box2poly
from polytope import Polytope
from polytope_oracles.samplers import SphereBasedSampler
from dataclasses import dataclass, field
from typing import List, Iterable
from timeit import default_timer as timer
from collections import defaultdict
import pandas as pd
import pickle

import os
from PolyRound.api import PolyRoundApi
from PolyRound.static_classes.lp_utils import ChebyshevFinder
from PolyRound.settings import PolyRoundSettings
from pathlib import Path


In [57]:
@dataclass
class Query:
    point: np.ndarray
    inside: bool
    
    @property
    def get_point(self):
        return point


@dataclass
class PolytopeQuery:
    polytope: HPolytope
    queries: List[Query] = field(default_factory=list)
    
    @property
    def dimension(self):
        return polytope.dimension()
    
    def __getstate__(self):
        state = self.__dict__.copy()
        state['A'] = self.polytope.A()
        state['b'] = self.polytope.b()
        del state['polytope']
        return state
    
    def __setstate__(self, state):
        A = state.pop('A')
        b = state.pop('b')
        self.__dict__.update(state)
        self.polytope = HPolytope(A, b)

Define experimentation protocol

In [58]:
def is_in(point: np.ndarray, polytope: HPolytope, loop: bool = True):
    if loop:
        for i in range(polytope.A().shape[0]):
            h = polytope.A()[i, :]
            if h.dot(point) > polytope.b()[i]:
                return False
        return True
    else:
        return (polytope.A().dot(point) <= polytope.b()).all()

In [59]:
def is_in_polytope(point: np.ndarray, polytope):
    return point in polytope

In [60]:
from tqdm.auto import tqdm


def run_experiments(polytope_queries: Iterable[PolytopeQuery], 
                    internal_point,
                    **membership_init_args) :
    results = {"time": [], "success": []}

    for polytope_query in polytope_queries:
        n = polytope_query.polytope.A().shape[0]
        d = polytope_query.polytope.dimension()

        print(f'n={n} -- d={d}')

        tic = timer()
        membership = Membership(polytope = polytope_query.polytope, internal_point = internal_point)
        membership.initialize(**membership_init_args)
        toc = timer()
        preprocessing_time = toc - tic
        
        for query in tqdm(polytope_query.queries):
            #tic = timer()
            #response = is_in(query.point, polytope_query.polytope, loop=True)
            #results['time'].append({"n": n, "d": d, "elapsed": timer() - tic, "type": "brute force loop"})
            #results['success'].append({"n": n, "d": d, "success": response == query.inside, "type": "brute force loop", "actual": query.inside})

            poly = Polytope(polytope_query.polytope.A(),polytope_query.polytope.b()) 
            tic = timer()
            response = is_in(query.point, polytope_query.polytope, loop=False)
            #response = is_in_polytope(query.point, poly)
            results['time'].append({"n": n, "d": d, "elapsed": timer() - tic, "type": "brute force vectorized"})
            results['success'].append({"n": n, "d": d, "success": response == query.inside, "type": "brute force vectorized", "actual": query.inside})

            tic = timer()
            response = membership.contains(query.point)
            results['time'].append({"n": n, "d": d, "elapsed": timer() - tic, "type": "membership"})
            results['success'].append({"n": n, "d": d, "success": response == query.inside, "type": "membership", "actual": query.inside})
    return pd.DataFrame(results['time']), pd.DataFrame(results['success']), preprocessing_time

# Hyper-cube experiments

## Preparation

For a number of different dimensions, generate cubes of varying side sizes, so that we can measure the effect of absolute numbers in our method

In [39]:
side_sizes = [100000] # [1, 10, 100, 1000, 10000]
dimensions = [100]# [10, 50, 100, 200, 500, 1000, 2000, 5000, 10000]

cubes_by_side = defaultdict(list)
for side_size in side_sizes:
    for d in dimensions:
        box = box2poly([(- side_size / 2, side_size / 2)] * d)
        cubes_by_side[side_size].append(HPolytope(box.A, box.b))

Define hyper-cube sampling method. For each dimension we take a number of values uniformly distributes within the bounds of each cube.

In [40]:
def sample_cube(cube: HPolytope, n_points: int = 100, seed: int = 42):
    generator = np.random.default_rng()
    sides = np.multiply(cube.A(), cube.b()[:, np.newaxis])
    points = [list() for _ in range(n_points)]
    for side in range(sides.shape[1]):
        for i, value in enumerate(generator.uniform(np.min(sides[:, side]), np.max(sides[:, side]), n_points)):
            points[i].append(value)
    return [Query(p, True) for p in points]

In [41]:
polytope_queries_by_side = {}
for side in cubes_by_side:
    polytope_queries_by_side[side] = [PolytopeQuery(polytope, sample_cube(polytope)) for polytope in cubes_by_side[side]]

In [42]:
with open("./cube_queries.pickle", "wb") as fout:
    pickle.dump(polytope_queries_by_side, fout)

In [43]:
! ls -lh cube_queries.pickle

-rw-rw-r-- 1 vissarion vissarion 346K Φεβ  18 17:46 cube_queries.pickle


# Experiment

In [26]:
with open("./cube_queries.pickle", "rb") as fin:
    polytope_queries_by_side = pickle.load(fin)

In [27]:
results = []
for side, polytopes in polytope_queries_by_side.items():
    results.append(run_experiments(polytopes))

n=200 -- d=100
0.03633671801071614


  0%|          | 0/100 [00:00<?, ?it/s]

In [28]:
df_time = pd.concat([r[0] for r in results])
df_time_gb = df_time.groupby(["n", "d", "type"]).mean()
df_time_gb

elapsed
n   d   type                            
200 100 brute force vectorized  0.000659
        membership              0.000035

In [29]:
df_accuracy = pd.concat([r[1] for r in results])
df_accuracy_gb = df_accuracy.groupby(["n", "d", "type"]).sum()
df_accuracy_gb['success %'] = df_accuracy_gb["success"] / df_accuracy_gb["actual"] * 100
df_accuracy_gb

success  actual  success %
n   d   type                                              
200 100 brute force vectorized      100     100      100.0
        membership                  100     100      100.0

# Random Polytopes

In [79]:
def random_polytope(d):
    n=10000*d
    A = np.zeros((n,d))
    for j in range(n):
        normal = 0
        p = np.zeros((d))
        for i in range(d):
            p[i] = np.random.normal()
            normal += p[i] * p[i]
        normal = 1 / np.sqrt(normal);
        p *= normal;
        A[j,:] = p
    
    b = np.array([10.0]*n)
    P = HPolytope(A,b)
    return P

In [7]:
class SphereBasedSampler:

    def __init__(self, mu: float = 0.0, sigma: float = 1.0, seed: int = 42):
        self.generator = np.random.default_rng(seed)
        self.mu = mu
        self.sigma = sigma
        self.seed = seed

    def generate(self, n: int, d: int) -> HPolytope:
        """
        :param n: Number of facets
        :param d: The target dimension
        :return: a generated polytope
        """

        points = self.generator.normal(self.mu, self.sigma, (n, d))
        norms = np.linalg.norm(points, axis=1, keepdims=True)
        points = points / norms

        return HPolytope(np.array(points), np.ones(n))

In [8]:
dimensions = [10]# [10, 50, 100, 200, 500, 1000, 2000, 5000, 10000]

polytopes = defaultdict(list)
for d in dimensions:
    #polytopes[0].append(random_polytope(d))
    sampler = SphereBasedSampler()
    polytopes[0].append(sampler.generate(1000*d, d))

    #polytopes[0].append(random_polytope(d))
    
#print(polytopes[0][0].A())
#print(polytopes[0][0].b())



In [9]:
 #cube = box2poly([(-2, 2 )] * 100)
 #print(cube.A)
 #print(cube.b)
 #cube_volesti = HPolytope(cube.A,cube.b)
# print (cube_volesti.A(),cube_volesti.b())


In [10]:
# samples = polytopes[0][0].generate_samples(walk_len = 10, number_of_points = 10, 
#                                  number_of_points_to_burn = 1, 
#                                  boundary = False, cdhr = True, rdhr = False, 
#                                  gaussian = False, set_L = False, billiard = False, 
#                                  ball_walk = False, a = 0, L = 0)
# samples
#P = cube_volesti
#max_ball = fast_inner_ball(P.A(), P.b()) 
#samples = P.generate_samples(cdhr=True, radius=max_ball[1],inner_point=max_ball[0]) 

#samples

In [144]:
def sample_polytope(P: HPolytope, n_points: int = 100, seed: int = 42):
    #max_ball = fast_inner_ball(P.A(), P.b()) 
    #print(P.A())
    #samples = P.generate_samples(walk_len = 20*P.dimension(), number_of_points = n_points, 
    #                             cdhr=True, radius=max_ball[1],inner_point=max_ball[0]) 
    samples =  PolytopeSampler.sample_from_polytope(P.A(), P.b(),15)
    samples = np.transpose(samples)
    #print(samples)
    print(samples.shape)
    return [Query(p, True) for p in samples]


In [12]:
polytope_queries = {}
polytope_queries = [PolytopeQuery(polytope, sample_polytope(polytope)) for polytope in polytopes[0]]
#polytope_queries = [PolytopeQuery(cube_volesti, sample_polytope(cube_volesti))]
#print(polytope_queries)

phase 1: number of correlated samples = 100, effective sample size = 59
[5]total ess 59: number of correlated samples = 100
(100, 10)


[5]maximum marginal PSRF: 1.01842


In [13]:
with open("./random_queries.pickle", "wb") as fout:
    pickle.dump(polytope_queries, fout)

In [14]:
! ls -lh random_queries.pickle

-rw-rw-r-- 1 vissarion vissarion 872K Φεβ  21 19:10 random_queries.pickle


In [15]:
with open("./random_queries.pickle", "rb") as fin:
    polytope_queries = pickle.load(fin)

In [16]:

results = list()
params = [{},  
          dict(m=300, max_m0=300, ef_construction=300)]
for param in params:
    results.append(run_experiments(polytope_queries, **param))


n=10000 -- d=10


  0%|          | 0/100 [00:00<?, ?it/s]

n=10000 -- d=10


  0%|          | 0/100 [00:00<?, ?it/s]

In [17]:
for r, param in zip(results, params):
    print(f'Params: {param}')
    df_time = pd.DataFrame(r[0])
    df_time_gb = df_time.groupby(["n", "d", "type"]).mean()
    print(df_time_gb.head(100))
# fig =  px.scatter_3d(df_time_gb.reset_index(), x='n', y='d', z='elapsed', color='type')
# fig.show()

Params: {}
                                  elapsed
n     d  type                            
10000 10 brute force vectorized  0.000129
         membership              0.000055
Params: {'m': 300, 'max_m0': 300, 'ef_construction': 300}
                                  elapsed
n     d  type                            
10000 10 brute force vectorized  0.000130
         membership              0.000144


In [19]:
for r, param in zip(results, params):
    print(f'Params: {param}')
    df_accuracy = pd.DataFrame(r[1])
    df_accuracy_gb = df_accuracy.groupby(["n", "d", "type"]).sum()
    df_accuracy_gb['success %'] = df_accuracy_gb["success"] / df_accuracy_gb["actual"] * 100
    print(df_accuracy_gb.head(100))

Params: {}
                                 success  actual  success %
n     d  type                                              
10000 10 brute force vectorized      100     100      100.0
         membership                    4     100        4.0
Params: {'m': 300, 'max_m0': 300, 'ef_construction': 300}
                                 success  actual  success %
n     d  type                                              
10000 10 brute force vectorized      100     100      100.0
         membership                   68     100       68.0


## Bio polytopes

In [61]:
models = ["e_coli_core","iLJ478","iSB619","iHN637","iJN678","iNF517","iJN746","iAB_RBC_283","iJR904","iAT_PLT_636","iSDY_1059","iAF1260","iEC1344_C","iJO1366","iBWG_1329","iML1515","RECON1","Recon2","Recon3D_latest"]
model = "RECON1"

In [62]:
polytope_samples_list = []
for model in models:
    with open("/home/vissarion/Downloads/dingo_samples/dingo_samples_" + model + ".pckl", "rb") as fin:
        polytope_samples = pickle.load(fin)
    #print(polytope_samples)
    polytope_samples = np.transpose(polytope_samples)
    #print(polytope_samples.shape)
    polytope_samples = polytope_samples[:100,]
    #print(polytope_samples.shape)
    polytope_samples_list.append(polytope_samples)
#polytope_samples_list[0]

In [134]:
polytopes = []
i=0
for model in models:
    with open("/home/vissarion/Downloads/simpl_transf_polytopes/simpl_transf_" + model + ".xml.pckl", "rb") as fin:
        polytope_xml = pickle.load(fin)
    print(i," ",model,end=" ")
    #print(polytope_xml[0])
    #print(polytope_xml[0].A.values)
    #print(polytope_xml[0].b.values)
    #print(polytope_xml[0].A.values.shape)
    #print(polytope_xml[0].b.values.shape)
    polytope = HPolytope(polytope_xml[0].A.values, polytope_xml[0].b.values*1000)
    #print(polytope.dimension())
    #print(np.dot(polytope_samples_list[i][0,],polytope_xml[0].A.values[0,]))
    #print(np.dot(polytope_samples[1,],polytope_xml[0].A.values[0,]))
    #is_in_polytope(polytope_samples[1,], polytope)
    print(is_in(polytope_samples_list[i][15,], polytope, loop=False),polytope.dimension())
    #print(is_in(np.zeros(polytope.dimension()), polytope, loop=False))
    polytopes.append(polytope)
    i=i+1

0   e_coli_core True 24
1   iLJ478 True 59
2   iSB619 True 83
3   iHN637 True 88
4   iJN678 True 92
5   iNF517 False 106
6   iJN746 False 122
7   iAB_RBC_283 True 130
8   iJR904 True 232
9   iAT_PLT_636 True 289
10   iSDY_1059 True 509
11   iAF1260 True 524
12   iEC1344_C False 578
13   iJO1366 False 582
14   iBWG_1329 False 613
15   iML1515 False 635
16   RECON1 False 931
17   Recon2 False 2430
18   Recon3D_latest False 5335


In [135]:
print(polytope_samples_list[0].shape)
print(polytope_samples_list[0][0,])
print(polytope_samples_list[0][1,])
print((polytope_samples_list[0][0,]+polytope_samples_list[0][1,])/2)
k=5
p_average = np.add.reduce(polytope_samples_list[k])/polytope_samples_list[k].shape[0]
#print(p_average)
print(is_in(p_average, polytopes[k], loop=False),polytope.dimension())

(100, 24)
[ -31.53314  -30.44121   72.98957   53.42099  -16.78    -102.38669
  -51.75637  121.85527  -32.5836  -106.62539  -46.3613   -16.53298
  -41.35618  133.86849   98.50333   15.09347  -96.24492  -11.74736
   21.57196  -95.08398  -23.19323  -83.35789  158.79595   28.81337]
[ -32.366    -34.39187   85.62878   59.43796  -22.46249 -109.29782
  -64.30022  136.90844  -36.99438 -113.9118   -61.49462  -13.90454
  -45.24147  152.98042   98.56325    3.38142 -119.62977   -2.2671
   39.94361  -87.1155   -36.32064 -102.76075  175.14722   32.67713]
[ -31.94957  -32.41654   79.30918   56.42947  -19.62125 -105.84226
  -58.0283   129.38185  -34.78899 -110.2686   -53.92796  -15.21876
  -43.29883  143.42446   98.53329    9.23745 -107.93735   -7.00723
   30.75778  -91.09974  -29.75694  -93.05932  166.97158   30.74525]
False 5335


In [161]:
k=3
polytope_queries = {}
polytope_queries = [PolytopeQuery(polytopes[k], [Query(p, is_in(p, polytopes[k], loop=False)) for p in polytope_samples_list[k]])]
polytope_queries = [PolytopeQuery(polytopes[k], sample_polytope(polytopes[k]))]
#print(polytope_queries[0].polytope.A().shape)
#print(polytope_queries[0].queries)



phase 1: number of correlated samples = 1800, effective sample size = 14, ratio of the maximum singilar value over the minimum singular value = 4154.76
phase 2: number of correlated samples = 100, effective sample size = 3
[5]total ess 17: number of correlated samples = 1900
(1900, 88)


[5]maximum marginal PSRF: 1.05683


In [162]:
results = list()
p_average = np.add.reduce(polytope_samples_list[k])/polytope_samples_list[k].shape[0]
params = [{},  
          dict(m=1000, max_m0=2000, ef_construction=2000)]
for param in params:
    results.append(run_experiments(polytope_queries, p_average, **param))


n=121 -- d=88


  0%|          | 0/1900 [00:00<?, ?it/s]

n=121 -- d=88


  0%|          | 0/1900 [00:00<?, ?it/s]

In [163]:
for r, param in zip(results, params):
    print(f'Params: {param}')
    df_time = pd.DataFrame(r[0])
    df_time_gb = df_time.groupby(["n", "d", "type"]).mean()
    print(df_time_gb.head(100))

Params: {}
                                elapsed
n   d  type                            
121 88 brute force vectorized  0.000011
       membership              0.000016
Params: {'m': 1000, 'max_m0': 2000, 'ef_construction': 2000}
                                elapsed
n   d  type                            
121 88 brute force vectorized  0.000011
       membership              0.000024


In [164]:
for r, param in zip(results, params):
    print(f'Params: {param}')
    df_accuracy = pd.DataFrame(r[1])
    df_accuracy_gb = df_accuracy.groupby(["n", "d", "type"]).sum()
    df_accuracy_gb['success %'] = df_accuracy_gb["success"] / df_accuracy_gb["actual"] * 100
    print(df_accuracy_gb.head(100))

Params: {}
                               success  actual  success %
n   d  type                                              
121 88 brute force vectorized     1900    1900      100.0
       membership                    0    1900        0.0
Params: {'m': 1000, 'max_m0': 2000, 'ef_construction': 2000}
                               success  actual  success %
n   d  type                                              
121 88 brute force vectorized     1900    1900      100.0
       membership                    0    1900        0.0


# Old

In [ ]:
d=100

In [14]:
box_out = box2poly([[-10001, 10001]] * d)
p_out = HPolytope(box_out.A, box_out.b)

In [15]:
m = Membership(polytope=p_out)

In [16]:
query = [0.01]*d

In [21]:
m.create_n2_nn(m=15)
m.contains_n2(query), m._index.search_by_vector(query, 1)

(False, [200])

In [9]:
m.create_hnsw_nn(m=5)
m._index.knn_query(query, k=1)

(array([[114]], dtype=uint64), array([[4.0008e+08]], dtype=float32))

(array([[9]], dtype=uint64), array([[4.0008e+08]], dtype=float32))

In [8]:
m.create_annoy_nn(5000)

TypeError: an integer is required (got type str)

In [39]:
m._index.search_by_vector([0.01]*d, 1, include_distances=True)

[(40, 0.8999999761581421)]

In [9]:
max_ball = fast_inner_ball(p.A(), p.b())
samples = p.generate_samples(walk_len=1, number_of_points=1000, radius=max_ball[1], inner_point=max_ball[0])

max_ball = fast_inner_ball(p_out.A(), p_out.b())
samples_out = p_out.generate_samples(walk_len=1, number_of_points=1000, boundary=True, radius=max_ball[1], inner_point=max_ball[0])

In [2]:
sampler = SphereBasedSampler()

In [3]:
d = 100

In [6]:
p = sampler.generate(d * 100, d)

In [ ]:
samples_out[0, :]

array([-1868.93666, -4586.7742 ,   514.71467,   576.38669,  3972.54032,
       -6774.19961, -3406.26128,  4121.20468,   528.16947,  1398.48769,
        -491.28975, -2680.04234, -2122.82087,  2996.4754 , -2513.2211 ,
       -1099.47762,  3685.66175,  -955.48015, -4364.11833,  4020.36754,
       -4421.60333,  4363.11061,  1480.50823, -5160.50437,    86.5002 ,
         326.42997, -4881.73193,   -94.8487 ,  3125.00804, -2779.26511,
       -3233.06436, -1533.41501, -2296.89104, -1038.61199,  2221.74466,
         114.91392,  2311.06249, -1522.20762,  -256.73503,  1350.9412 ,
        9144.16378,  1701.41608, -2791.53404, -4685.89362, -3543.41628,
       -4866.84575,  3819.95207, -2809.14988,  8556.10131, -1736.29506,
        5702.03686,  -647.9397 , -3966.69331,  2899.04392,  1127.25715,
        1588.8931 ,  1142.11892,  4142.42772, -2832.8605 , -1567.92027,
        4635.98342,   749.40934, -2319.70158,  1121.18358,  2014.69571,
        3987.81428, 10001.     ,  -703.74761, -3474.71281, -2777

In [ ]:
m = Membership(polytope=p)
m.create_annoy_nn(25000)

In [ ]:
from timeit import default_timer as timer

success = 0
in_c = 0

bf_t = 0
m_t = 0
for i in range(samples.shape[0]):
    tic = timer()
    inside = m.contains(samples[i, :])
    m_t += timer() - tic
    tic = timer()
    if is_in(p, samples[i, :]):
        in_c += 1
    bf_t += timer() - tic
#     print(f"{samples[i, :]} -- inside: {inside}")
    if inside:
        success += 1
success, in_c, m_t / samples.shape[0], bf_t / samples.shape[0], m_t, bf_t

(841,
 1000,
 0.01833995155838784,
 0.02667680139215372,
 18.33995155838784,
 26.67680139215372)

- memory leak sto acc. billiard walk gia n = 100000, d = 100
- kivoi se poli megales diastaseis xwris sampling sigrisi me b.f. (ws pros n, d)
- random polytopes me megalitero radius sti bala kai sigrisi me b.f. (ws pros n, d)
- recon polytopes

In [ ]:
success = 0
in_c = 0

bf_t = 0
m_t = 0

for i in range(samples_out.shape[0]):
    inside = m.contains(samples_out[i, :])
    if is_in(p, samples_out[i, :]):
        in_c += 1
#     print(f"{samples[i, :]} -- inside: {inside}")
    if not inside:
        success += 1
success, in_c

IndexError: Vector has wrong length (expected 500, got 1000)

In [ ]:
m = Membership(polytope=p_out)
m.create_annoy_nn(10000)

In [ ]:
success = 0
for i in range(samples.shape[0]):
    inside = m.contains(samples[i, :])
#     print(f"{samples[i, :]} -- inside: {inside}")
    if inside:
        success += 1
success

993